# Week3: Motif Finding

In [1]:
def parse_file(file_name):
    f = open(file_name, 'r')
    Text = f.read().splitlines()
    f.close()
    return Text

In [2]:
def first_symbol(pattern):
    return pattern[0:1]

def suffix(pattern):
    return pattern[1:]

def hamming_distance(text1, text2):
    hamm_dist = 0
    for i in range(0,len(text1)):
        if (text1[i] != text2[i]):
            hamm_dist += 1
    return hamm_dist


In [3]:
def find_neighbors(pattern, d):
    nbs = {'A', 'C', 'T', 'G'}
    if d == 0:
        return pattern
    if len(pattern) == 1:
        return nbs
    neighborhood = set()
    suffix_pattern = suffix(pattern)
    suffix_neighbors = find_neighbors(suffix_pattern, d)
    for text in suffix_neighbors:
        if hamming_distance(suffix(pattern), text) < d:
            for nb in nbs:
                neighborhood.add(nb + text)
        else:
            neighborhood.add(first_symbol(pattern) + text)
    return neighborhood

In [4]:
def motif_enumeration(dna, k ,d):
    kmer_list = [set() for _ in dna] # list of sets
    for pos, pattern in enumerate(dna):
        for k_pos in range(len(pattern) - k + 1):
            k_mer = pattern[k_pos:k_pos+k]
            neighbors = find_neighbors(k_mer, d)
            kmer_list[pos].update(neighbors)
    patterns = kmer_list[0]
    for kset in kmer_list:
        patterns = patterns.intersection(kset)
    return patterns

text = parse_file('dataset_156_8.txt')[1]
dna = text.split(' ')
print(*motif_enumeration(dna, 5, 2))

CTTCT AGCTA CTTAG CTGTC ACGTT TTCTA ACCCC ATAGA GCCTG TACGA TTTGT TCGCC ACTTG ACACC TAAGT TACGC TGCAC GGATT AAGTT GGAAG TCGTC GATTA ACCGT CCTTG TCCCA CTTAC CACTT CTGCG CAGCC TGGAA AGCTT AAGTG CTCGT CCAGC TCGCT CATAG GTAAA GAAGG TGATC GGTAA ATCAC AGGTC AACGT TAGGT TACTT TCAGA TAACT CTGTG ACATG GTGCA ATGGA GGACT TGCAA GACCT TGTCT CATTT AGCCG ATCCC AGAGC CTCTG TCAGT TGCTG CCCAT GCATT AGCCC TGGTT GTCTG AATTG TATCG TTGCT GATTT GCGTA TAGAA TGACT TCCTG TGCCT TGCGT ACCAC GGTTT TCCTT CACTG GTCTC GCAGT GTACA CCGTT TTGAA GGTTA GGATG AACGC CACCC CTCAG AGATT ACTGC TAGCT GAATA TAGAC AGCCT TCCAG ATCGT GCTTC TGCAT TCCTC CGGTT TAGGG CTTTC GTGTC CGGAA TGAAC CTACC ATAGG CACAT CTAGT ATCTT TACTC ACCCA CTTTT ACTTT CCCTA GAGCC TCACC TGCCC CGTTT CACTC TCGTG AAACC AGTGA GACAT GCGTT GATCG CTGAT ATGTC GTTGG TAGGC GGTAG TTAGC ACATT GGCTT ATTAG CTTCA ATCGC ATGTA CCTAT AGTAT ATGGT GCTCT GCCTT CTACG GTTCG TCGCG TTGAG TGCGA TAATT ACTTC GGTTC CCATC ATCGA GGTGT GATGT TTCTG ATGCT ACGTC AAGGT CTATC TTCAC TCCAC TGTAC GCAT

# Median String Problem

In [5]:
def distance_between_pattern_and_strings(pattern, dna):
    k = len(pattern)
    distance = 0
    for string in dna:
        hamm_dist = float('inf')
        for k_pos in range(len(string) - k + 1):
            k_mer = string[k_pos:k_pos+k]
            if hamm_dist > hamming_distance(pattern, k_mer):
                hamm_dist = hamming_distance(pattern, k_mer)
        distance = distance + hamm_dist
    return distance

In [6]:
text = parse_file('dataset_5164_1.txt')
pattern = text[0]
dna = text[1].split(' ')
distance_between_pattern_and_strings(pattern, dna)

89

In [7]:
def all_strings(d):
    nbs = {'A', 'C', 'T', 'G'}
    if d == 1:
        return nbs
    strings = set()
    suffix_strings = all_strings(d-1)
    for string in suffix_strings:
        for nb in nbs:
            strings.add(nb + string)
    return strings
all_strings(2)

{'AA',
 'AC',
 'AG',
 'AT',
 'CA',
 'CC',
 'CG',
 'CT',
 'GA',
 'GC',
 'GG',
 'GT',
 'TA',
 'TC',
 'TG',
 'TT'}

In [8]:
def median_string(dna, k):
    distance = k*len(dna)
    median = []
    patterns = all_strings(k)
    for pattern in patterns:
        if distance > distance_between_pattern_and_strings(pattern, dna):
            distance = distance_between_pattern_and_strings(pattern, dna)
            median.append(pattern)
    return median

In [207]:
text = 'CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG'
dna = text.split(' ')
median_string(dna, 7)

['GGACTCA', 'AATCCTA']

# Profile-most Probable k-mer Problem

In [112]:
def find_profile_most_probable_kmer(Dna, k, profile):
    maxProbability = -1
    # Compute the probability of the each k-mer, store it if it's currently a maximum.
    for i in range(len(Dna)-k+1):
        # Get the current probability.
        currentProbability = 1
        currentProbability = compute_probability(Dna[i:i+k], profile)
        # Check for a maximum.
        if currentProbability > maxProbability:
            maxProbability = currentProbability
            mostProbable = Dna[i:i+k]

    return mostProbable

In [119]:
def compute_probability(k_mer, profile_matrix):
    nb_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    prob = 1.0
    nb_index = 0
    for i, nt in enumerate(k_mer):
        prob *= profile_matrix[i][nb_dict[nt]]
    return prob

# Greedy Motif Search with Pseudocounts

The basic idea of the greedy motif search algorithm is to find the set of motifs across a number of DNA sequences that match each other  most closely.

In [120]:
def greedy_motif_search(dna, k, t):
    best_motifs = [string[0:k] for string in dna]
    first_dna_string = dna[0]
    motifs = [0]*t
    profile = [[0.0]*4 for i in range(k)]
    for k_pos in range(len(dna[0]) - k + 1):
        motifs = [dna[0][k_pos:k_pos+k]]
        for i in range(1,t):
            profile = update_profile_and_add_pseudocouns(profile, motifs)
            most_probable_kmer = find_profile_most_probable_kmer(dna[i], k, profile)
            motifs.append(most_probable_kmer)
        if score(motifs, k) < score(best_motifs, k):
            best_motifs = motifs[:]            
    return best_motifs

In [121]:
def score(motifs, k):
    score = 0
    consensus = find_consensus_motif(motifs, k)
    for motif in motifs:
        score += hamming_distance(motif, consensus)
    return score    

In [122]:
def find_consensus_motif(motifs, k):
    consensus = ''
    colums = [0]*k
    for i in range(k):
        score_col = 0
        nb_dict = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        colums[i] = [motif[i] for motif in motifs]
        for nb in colums[i]:
            nb_dict[nb] +=1
        max_count = max(nb_dict.values())
        max_nb = [nb for nb in nb_dict.keys() if nb_dict[nb] == max_count]
        consensus += max_nb[0]
    return consensus

In [194]:
def update_profile_and_add_pseudocouns(profile, motifs):
    pseudocount = 1.0
    columns = [''.join(seq) for seq in zip(*motifs)]
    return [[float(col.count(nuc)+pseudocount) / float(len(col)) for nuc in 'ACGT'] for col in columns]

In [195]:
text = 'GGCGTTCAGGCA AAGAATCAGTCA CAAGGAGTTCGC CACGTCAATCAC CAATAATATTCG'
dna = text.split(' ')
#text = parse_file('dataset_159_5.txt')
#dna = [x for x in text[1].split(' ')]
print(*greedy_motif_search(dna, 3, 5))

TTC ATC TTC ATC TTC


In [196]:
motifs = ['GGC', 'AAG', 'AAG', 'CAC', 'CAA']
score(motifs, 3)

7

# Week 4: Randomized Motif Search

Random search resulted in a biased profile, because the genome is not random

In [197]:
import random
def get_random_motifs_from_dna(dna, k):
    motifs = []
    for genome in dna:
        idx = random.randrange(0, len(genome) - k + 1)
        motifs.append(genome[idx:idx+k])
    return motifs

In [219]:
def randomized_motif_search(dna, k, t):
    motifs = get_random_motifs_from_dna(dna, k)
    best_motifs = motifs[:]
    profile = [[0.0]*4 for i in range(k)]
    while True:
        profile = update_profile_and_add_pseudocouns(profile, motifs)
        most_probable_motifs = []
        for i in range(0,t):
            most_probable_kmer = find_profile_most_probable_kmer(dna[i], k, profile)
            most_probable_motifs.append(most_probable_kmer)
            motifs = most_probable_motifs[:]
        if score(motifs, k) < score(best_motifs, k):
            best_motifs = motifs[:]  
        else:
            return best_motifs

In [220]:
text = parse_file('dataset_161_5.txt')
dna = [x for x in text[1].split(' ')]
k = 15
t = 20
best_motifs = randomized_motif_search(dna, k, t)
for _ in range(1000):
    last_motifs = randomized_motif_search(dna, k, t)
    if score(last_motifs, k) < score(best_motifs, k):
        best_motifs = last_motifs[:]  
print(*best_motifs)

TCACCCTACAGTCTC TACCCATACAGCCTC TACTGCGTCAGCCTC TACTGTTAGTACCTC TACTGTGCTAGCCTC TACTGTTACGTGCTC AGGTGTTACAGCCTC TACTTGAACAGCCTC GACTGTTACAGCCGA CGCTGTTACAGCCTG TACTGTGTTAGCCTC TATGCTTACAGCCTC TACTGTTGACGCCTC TACTGGGGCAGCCTC TACTGTTACAGTGGC TACTGTTACATTTTC TGTAGTTACAGCCTC TACATCTACAGCCTC TACTGTTACAGCACG TACTCCCACAGCCTC
